## 54단계: 드롭아웃과 테스트 모드

> 이번 단계에서는 DeZero에 드롭아웃을 추가합니다. 드롭아웃을 적용하려면 학습 시와 테스트 시의 처리 로직을 달리해야 합니다. 그래서 학습 단계인지 테스트 단계인지 구별하는 구조를 만들겠습니다.

### 54.1 드롭아웃이란

드롭아웃은 학습 시에 은닉층 뉴런을 무작위로 골라 삭제한다.

<img src="images/그림 54-1.png" width=400/>

이 동작은 아래처럼 구현할 수 있다.

In [1]:
import numpy as np

dropout_ratio = 0.6
x = np.ones(10)

mask = np.random.rand(10) > dropout_ratio
y = x * mask

테스트 시에는 모든 뉴런을 사용하면서도 앙상블 학습처럼 동작하게끔 '흉내' 내야 한다.

In [2]:
# 학습 시
mask = np.random.rand(10) > dropout_ratio
y = x * mask

# 테스트 시
scale = 1 - dropout_ratio
y = x * scale

### 54.2 역 드롭아웃

역 드롭아웃은 스케일 맞추기를 '학습할 때' 수행한다.

In [3]:
# 학습 시
scale = 1 - dropout_ratio
mask = np.random.rand(10) > dropout_ratio
y = x * mask

# 테스트 시
y = x

역 드롭아웃은 테스트 속도가 약간 향상되고, 학습 시 dropout_ratio를 동적으로 변경할 수 있다. \
이런 이점들 때문에 많은 딥러닝 프레임워크에서 역 드롭아웃 방식을 채용하고 있다.

### 54.3 테스트 모드 추가

In [4]:
## dezero/core.py

class Config:
    enable_backprop = True
    train = True

def using_config(name, value):
    ...

def test_mode():
    return using_config('train', False)

### 54.4 드롭아웃 구현

In [5]:
# dezero/functions.py
import dezero
from dezero.core import as_variable
from dezero import cuda

def dropout(x, dropout_ratio=0.5):
    x = as_variable(x)
    
    if dezero.Config.train:
        xp = cuda.get_array_module(x)
        mask = xp.random.rand(*x.shape) > dropout_ratio
        scale = xp.array(1.0 - dropout_ratio).astype(x.dtype)
        y = x * mask / scale
        return y
    else:
        return x

In [6]:
import numpy as np
from dezero import test_mode
import dezero.functions as F

x = np.ones(5)
print(x)

# 학습 시
y = F.dropout(x)
print(y)

# 테스트 시
with test_mode():
    y = F.dropout(x)
    print(y)

[1. 1. 1. 1. 1.]
Variable([0. 0. 2. 2. 0.])
Variable([1. 1. 1. 1. 1.])
